airdoorのスクレイピング

In [66]:
import requests # requests機能をインポートします。

In [67]:
url = 'https://airdoor.jp/list?si=d-131091'
res = requests.get(url) #リクエストしたデータをresに代入します。
result = res.text #resのtextにサイトデータが格納されているので、それをresultに代入します。
print(result) # printでサイトデータを確認します。

<!DOCTYPE html><html lang="ja"><head><meta charSet="utf-8"/><meta name="twitter:card" content="summary"/><meta name="twitter:site" content="@airdoor_jp"/><meta name="twitter:creator" content="@airdoor_jp"/><meta property="og:type" content="website"/><meta property="og:site_name" content="エアドア"/><meta name="viewport" content="width=device-width,initial-scale=1.0,maximum-scale=1.0"/><title>東京都品川区の賃貸物件｜仲介手数料無料・初期費用安くするならエアドア</title><meta name="robots" content="index,follow"/><meta name="description" content="東京都品川区のリアルタイムな賃貸物件情報。仲介手数料無料・礼金なし・フリーレントなど初期費用が安い物件を取り揃えています。効率良く探したい方、お急ぎや忙しい方にもオススメ。"/><meta property="og:title" content="東京都品川区の賃貸物件｜仲介手数料無料・初期費用安くするならエアドア"/><meta property="og:description" content="東京都品川区のリアルタイムな賃貸物件情報。仲介手数料無料・礼金なし・フリーレントなど初期費用が安い物件を取り揃えています。効率良く探したい方、お急ぎや忙しい方にもオススメ。"/><meta property="og:url" content="https://airdoor.jp/list?si=d-131091"/><meta property="og:image" content="https://d700om8pmcpt7.cloudfront.net/buildings/images/4bf74fa7cdfc70a424dd3dfdd311f6ab.jpg"/

In [68]:
print(res.encoding) # res.encodingで文字エンコードを確認できます。

utf-8


In [69]:
from bs4 import BeautifulSoup # BeautifulSoupの機能をインポートします。
soup = BeautifulSoup(res.text,"html.parser") # BeautifulSoup(解析したいデータ,解析する方法)を指定し、解析したデータをsoupに代入します。

In [70]:
from retry import retry
import urllib
import time

In [71]:
# 複数ページの情報をまとめて取得
data_samples = []

page = 1  # 初期ページ
while True:
    # ページ情報
    soup = load_page(url.format(page))
    
    # スクレイピングの処理（省略）

    # 物件情報リストを指定
    mother = soup.find_all(class_='PropertyPanel_propertyPanel__MqCpF')
    
    # ページ内の物件ごとの処理
    for child in mother:
        # スクレイピングの処理（省略）

        # 物件情報と部屋情報をくっつける
        data_samples.append(data_home)
    
    # 次のページに進む
    page += 1

    # ここで適切な終了条件を設定する（例: 特定の条件が満たされたら終了）
    if not soup.find_all(class_='next_page'):
        # 次のページが存在しない場合は終了
        break

# リクエストがうまく行かないパターンを回避するためのやり直し
@retry(tries=3, delay=10, backoff=2)
def load_page(url):
    html = requests.get(url)
    soup = BeautifulSoup(html.content, 'html.parser')
    return soup

In [72]:
#各種データを取得
import re
import time

def extract_room_info(room_info):
    # 部屋番号、間取り、広さ、方角を正規表現で抽出
    room_matches = re.match(r'(\d+)号室 \/ (\S+) \/ (\d+(\.\d+)?)㎡ \/ (.+)', room_info)

    if room_matches:
        # room_numberのクレンジング処理
        room_number_raw = room_matches.group(1)
        room_number = str(int(room_number_raw))  # 半角数字に変換
        return room_number, room_matches.group(2), room_matches.group(3), room_matches.group(5)
    else:
        return "", "", "", ""

def extract_address_info(building_info):
    # 住所情報を正規表現で抽出
    address_matches = re.findall(r'(東京都|大阪府|京都府|などの都道府県の正規表現)(.*?)(\d+丁目|$)', building_info)

    if address_matches:
        prefecture = address_matches[0][0]
        city = address_matches[0][1].replace('の', '')  # 区市町村から不要な文字を削除
        address = address_matches[0][2]
        return prefecture, city, address
    else:
        return "", "", ""

# data_samplesを初期化
data_samples = []

# 分割するためにインポート
import re

# ページごとの処理
for page in range(1, max_page + 1):
    before = time.time()
    # ページ情報
    soup = load_page(url.format(page))
    # 物件情報リストを指定
    mother = soup.find_all(class_='PropertyPanel_propertyPanel__MqCpF')
    # 物件ごとの処理
    for child in mother:
        # 建物情報
        data_home = []
        # 建物名
        building_info = child.find(class_='PropertyPanelBuilding_buildingTitle__NbWmb').text
        building_info_parts = re.split(r'\s+', building_info)  # 空白で分割
        vacancy_info = building_info_parts[0] if building_info_parts else ""
        building_name = building_info_parts[1] if len(building_info_parts) > 1 else ""

        data_home.append(vacancy_info)
        data_home.append(building_name)

        # 家賃、管理費
        rent_info = child.find(class_='PropertyPanelRoom_rentPrice__HO4Jp').text
        rent_matches = re.match(r'(\d+,\d+)円 \((.+)\)', rent_info)
        rent = rent_matches.group(1) if rent_matches else ""
        management_fee = rent_matches.group(2) if rent_matches else ""

        # 1. rent の末尾に「円」を付ける
        rent += "円"

        # 2. management_fee から文頭の「管理費」を取り除く
        management_fee = management_fee.replace("管理費", "").replace("円", "")

        # 3. rent と management_fee を足して total_fee を項目として追加する
        total_fee = str(int(rent.replace("円", "").replace(",", "")) + int(management_fee.replace(",", "")))

        # 4. 管理費とtotal_feeにも円を付け足す
        management_fee += "円"
        total_fee += "円"

        data_home.append(rent)
        data_home.append(management_fee)
        data_home.append(total_fee)

        # 部屋情報（間取り）
        room_info = child.find(class_='is-ml5').text

        # 部屋情報を正規表現で抽出
        room_number, layout, area, direction = extract_room_info(room_info)

        # 最寄り駅のアクセス、住所、建物情報、築年数、階数
        building_info = child.find(class_='PropertyPanelBuilding_buildingInformationSection__AMRsh').text

        # 住所情報を正規表現で抽出
        prefecture, city, address = extract_address_info(building_info)

        # 正規表現を使って情報を抽出
        walk_time_matches = re.findall(r'徒歩(\d+)分', building_info)
        delivery_box_matches = re.findall(r'宅配ボックス', building_info)
        parking_matches = re.findall(r'駐車場あり', building_info)
        elevator_matches = re.findall(r'エレベーター', building_info)

        # 住所、徒歩、宅配ボックス、駐車場あり、エレベーターの情報を取得
        walk_time = walk_time_matches[0] if walk_time_matches else ""
        delivery_box = "〇" if delivery_box_matches else ""
        parking = "〇" if parking_matches else ""
        elevator = "〇" if elevator_matches else ""

        # 最寄り駅のアクセス、住所、建物情報、築年数、階数
        building_info2 = child.find(class_='PropertyPanelBuilding_buildingInformation__kRI1U is-pc-only').text

        # 築年数、階数、材質情報を正規表現で抽出
        built_year_matches = re.search(r'築(\d+年|新築)', building_info2)
        floors_matches = re.search(r'(\d+)階建', building_info2)
        material_matches = re.search(r'(鉄筋コンクリート|鉄骨造|木造|軽量鉄骨|その他)', building_info2)

        # 築年数、階数、材質情報を変数に格納
        built_year = built_year_matches.group(1) if built_year_matches else "新築"
        floors = floors_matches.group(1) if floors_matches else ""
        material = material_matches.group(1) if material_matches else ""

        # お得情報(項目)
        sale_info = child.find(class_='PropertyPanelRoom_isRed__HXtHO PropertyPanelRoom_isBold__hkYt7').text

        # お得情報(額面)
        sale_info2 = child.find(class_='PropertyPanelRoom_strikeText__N_r0P').text

        # お得情報(概要)を取得
        sale_info3_element = child.find(class_='PropertyPanelRoom_freerentTagBlock__n_YOv')
        sale_info3 = sale_info3_element.text if sale_info3_element else ""

        # お得情報(星)を取得
        stars_container = child.find(class_='InitialCostScore_starsContainer__q41Lt')

        # おすすめ度の星の数を数える
        recommended_stars = stars_container.find_all('span', class_='Stars_starSvg__NcvTr Stars_active__7B_Zp') if stars_container else []
        not_recommended_stars = stars_container.find_all('span', class_='Stars_starSvg__NcvTr Stars_inActive__u6vwG') if stars_container else []

        # おすすめ度の数をカウント
        recommended_count = len(recommended_stars)
        not_recommended_count = len(not_recommended_stars)

        # カラムに情報を追加
        data_home.extend([room_number, layout, area, direction, prefecture, city, address, walk_time, delivery_box, parking, elevator, built_year, floors, material, sale_info , sale_info2, sale_info3, recommended_count])

        # 物件情報と部屋情報をくっつける
        data_samples.append(data_home)


In [73]:
import pandas as pd
df = pd.DataFrame(data_samples)
df

,0,1,2,3,4,5,6,7,8,9,...,13,14,15,16,17,18,19,20,21,22
0,【空室95件】,Ｊ．ＧＲＡＮ,"109,000円","10,000円",119000円,105,1K,25.8,西,東京都,...,〇,,,新築,5,鉄筋コンクリート,仲介手数料無料,"119,900円",フリーレント2ヶ月(21.8万円お得),3
1,【空室2件】,ＫＤＡ,"106,000円","10,000円",116000円,502,1K,22.5,南西,東京都,...,〇,,〇,4年,14,鉄筋コンクリート,仲介手数料無料,"116,600円",フリーレント1ヶ月(10.6万円お得),3
2,【空室3件】,ＡＬＴＥＲＮＡ南大井,"99,000円","10,000円",109000円,402,1K,21.7,西,東京都,...,〇,,〇,1年,12,鉄筋コンクリート,仲介手数料無料,"108,900円",,2
3,【空室102件】,Ｊ．ＧＲＡＮ,"202,000円","20,000円",222000円,805,2LDK,40.9,南,東京都,...,〇,,,新築,9,鉄筋コンクリート,仲介手数料無料,"222,200円",フリーレント2ヶ月(27.2万円お得),1
4,【空室1件】,Bark,"110,000円","8,000円",118000円,303,1K,25.18,南西,東京都,...,〇,,〇,新築,4,鉄筋コンクリート,仲介手数料無料,"121,000円",,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,【空室3件】,Ｌａ,"104,000円","5,000円",109000円,403,1R,20.38,北西,東京都,...,〇,,,2年,4,鉄筋コンクリート,仲介手数料無料,"114,400円",フリーレント1ヶ月(10.4万円お得),3
196,【空室3件】,ウィンベルソロ大森第2,"68,000円","5,000円",73000円,503,1R,20.28,南,東京都,...,〇,,〇,32年,6,鉄筋コンクリート,"51,000円","74,800円",,1
197,【空室3件】,ライオンズフォーシア五反田,"162,000円","12,000円",174000円,601,1LDK,31.72,南西,東京都,...,〇,,,2年,15,鉄筋コンクリート,仲介手数料無料,"178,200円",,2
198,【空室1件】,パークシティ武蔵小山ザタワー,"250,000円","20,000円",270000円,718,1LDK,48.31,北西,東京都,...,〇,,〇,3年,41,鉄筋コンクリート,"88,000円","275,000円",,0


In [74]:
df = df.rename(columns = {0:'空室情報',1:'建物名',2:'家賃',3:'管理費',4:'合計金額',5:'部屋No',6:'間取り',7:'面積',8:'方角',9:'都道府県',10:'市区町村',11:'丁目',12:'駅徒歩',13:'宅配ボックス',14:'駐車場',15:'エレベーター',16:'築年情報',17:'物件階数',18:'材質',19:'お得情報',20:'値引金額',21:'追加セールスポイント',22:'おすすめ度'})
df

,空室情報,建物名,家賃,管理費,合計金額,部屋No,間取り,面積,方角,都道府県,...,宅配ボックス,駐車場,エレベーター,築年情報,物件階数,材質,お得情報,値引金額,追加セールスポイント,おすすめ度
0,【空室95件】,Ｊ．ＧＲＡＮ,"109,000円","10,000円",119000円,105,1K,25.8,西,東京都,...,〇,,,新築,5,鉄筋コンクリート,仲介手数料無料,"119,900円",フリーレント2ヶ月(21.8万円お得),3
1,【空室2件】,ＫＤＡ,"106,000円","10,000円",116000円,502,1K,22.5,南西,東京都,...,〇,,〇,4年,14,鉄筋コンクリート,仲介手数料無料,"116,600円",フリーレント1ヶ月(10.6万円お得),3
2,【空室3件】,ＡＬＴＥＲＮＡ南大井,"99,000円","10,000円",109000円,402,1K,21.7,西,東京都,...,〇,,〇,1年,12,鉄筋コンクリート,仲介手数料無料,"108,900円",,2
3,【空室102件】,Ｊ．ＧＲＡＮ,"202,000円","20,000円",222000円,805,2LDK,40.9,南,東京都,...,〇,,,新築,9,鉄筋コンクリート,仲介手数料無料,"222,200円",フリーレント2ヶ月(27.2万円お得),1
4,【空室1件】,Bark,"110,000円","8,000円",118000円,303,1K,25.18,南西,東京都,...,〇,,〇,新築,4,鉄筋コンクリート,仲介手数料無料,"121,000円",,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,【空室3件】,Ｌａ,"104,000円","5,000円",109000円,403,1R,20.38,北西,東京都,...,〇,,,2年,4,鉄筋コンクリート,仲介手数料無料,"114,400円",フリーレント1ヶ月(10.4万円お得),3
196,【空室3件】,ウィンベルソロ大森第2,"68,000円","5,000円",73000円,503,1R,20.28,南,東京都,...,〇,,〇,32年,6,鉄筋コンクリート,"51,000円","74,800円",,1
197,【空室3件】,ライオンズフォーシア五反田,"162,000円","12,000円",174000円,601,1LDK,31.72,南西,東京都,...,〇,,,2年,15,鉄筋コンクリート,仲介手数料無料,"178,200円",,2
198,【空室1件】,パークシティ武蔵小山ザタワー,"250,000円","20,000円",270000円,718,1LDK,48.31,北西,東京都,...,〇,,〇,3年,41,鉄筋コンクリート,"88,000円","275,000円",,0


In [99]:
# 重複が削除できていなさそうだったので、改めて特定の複数列で重複削除を指示
df = df.drop_duplicates(subset=['建物名', '合計金額', '面積','部屋No'])
df

,空室情報,建物名,家賃,管理費,合計金額,部屋No,間取り,面積,方角,都道府県,...,宅配ボックス,駐車場,エレベーター,築年情報,物件階数,材質,お得情報,値引金額,追加セールスポイント,おすすめ度
0,【空室95件】,Ｊ．ＧＲＡＮ,"109,000円","10,000円",119000円,105,1K,25.8,西,東京都,...,〇,,,新築,5,鉄筋コンクリート,仲介手数料無料,"119,900円",フリーレント2ヶ月(21.8万円お得),3
1,【空室2件】,ＫＤＡ,"106,000円","10,000円",116000円,502,1K,22.5,南西,東京都,...,〇,,〇,4年,14,鉄筋コンクリート,仲介手数料無料,"116,600円",フリーレント1ヶ月(10.6万円お得),3
2,【空室3件】,ＡＬＴＥＲＮＡ南大井,"99,000円","10,000円",109000円,402,1K,21.7,西,東京都,...,〇,,〇,1年,12,鉄筋コンクリート,仲介手数料無料,"108,900円",,2
3,【空室102件】,Ｊ．ＧＲＡＮ,"202,000円","20,000円",222000円,805,2LDK,40.9,南,東京都,...,〇,,,新築,9,鉄筋コンクリート,仲介手数料無料,"222,200円",フリーレント2ヶ月(27.2万円お得),1
4,【空室1件】,Bark,"110,000円","8,000円",118000円,303,1K,25.18,南西,東京都,...,〇,,〇,新築,4,鉄筋コンクリート,仲介手数料無料,"121,000円",,2
5,【空室1件】,SS.FLAT北品川,"105,000円","8,000円",113000円,301,1K,20.95,北東,東京都,...,〇,,〇,2年,14,鉄筋コンクリート,仲介手数料無料,"115,500円",,2
6,【空室1件】,リビオメゾン戸越銀座,"215,000円","15,000円",230000円,1302,2LDK,44.2,西,東京都,...,〇,〇,〇,2年,15,鉄筋コンクリート,仲介手数料無料,"236,500円",,1
7,【空室31件】,ＯＲＳＵＳ戸越銀座,"112,000円","10,000円",122000円,107,1R,25.14,北東,東京都,...,〇,,〇,新築,6,鉄筋コンクリート,仲介手数料無料,"123,200円",,2
8,【空室4件】,ディームス大井町フォレスト,"94,000円","10,000円",104000円,514,1R,20.91,西,東京都,...,〇,,〇,2年,4,鉄筋コンクリート,仲介手数料無料,"103,400円",フリーレント1ヶ月(9.4万円お得),3
9,【空室3件】,プラウドフラット戸越公園,"104,000円","8,000円",112000円,220,1K,22.32,北東,東京都,...,〇,〇,〇,5年,5,鉄筋コンクリート,仲介手数料無料,"114,400円",フリーレント1ヶ月(10.4万円お得),3


In [100]:
# drop_duplicates()で重複したデータを削除してくれます。
# drop_duplicates(inplace=True)とすることで、処理したデータを出力だけでなく、出力したデータを元のdfに代入して変更してくれます。
df.drop_duplicates(inplace=True)
df

,空室情報,建物名,家賃,管理費,合計金額,部屋No,間取り,面積,方角,都道府県,...,宅配ボックス,駐車場,エレベーター,築年情報,物件階数,材質,お得情報,値引金額,追加セールスポイント,おすすめ度
0,【空室95件】,Ｊ．ＧＲＡＮ,"109,000円","10,000円",119000円,105,1K,25.8,西,東京都,...,〇,,,新築,5,鉄筋コンクリート,仲介手数料無料,"119,900円",フリーレント2ヶ月(21.8万円お得),3
1,【空室2件】,ＫＤＡ,"106,000円","10,000円",116000円,502,1K,22.5,南西,東京都,...,〇,,〇,4年,14,鉄筋コンクリート,仲介手数料無料,"116,600円",フリーレント1ヶ月(10.6万円お得),3
2,【空室3件】,ＡＬＴＥＲＮＡ南大井,"99,000円","10,000円",109000円,402,1K,21.7,西,東京都,...,〇,,〇,1年,12,鉄筋コンクリート,仲介手数料無料,"108,900円",,2
3,【空室102件】,Ｊ．ＧＲＡＮ,"202,000円","20,000円",222000円,805,2LDK,40.9,南,東京都,...,〇,,,新築,9,鉄筋コンクリート,仲介手数料無料,"222,200円",フリーレント2ヶ月(27.2万円お得),1
4,【空室1件】,Bark,"110,000円","8,000円",118000円,303,1K,25.18,南西,東京都,...,〇,,〇,新築,4,鉄筋コンクリート,仲介手数料無料,"121,000円",,2
5,【空室1件】,SS.FLAT北品川,"105,000円","8,000円",113000円,301,1K,20.95,北東,東京都,...,〇,,〇,2年,14,鉄筋コンクリート,仲介手数料無料,"115,500円",,2
6,【空室1件】,リビオメゾン戸越銀座,"215,000円","15,000円",230000円,1302,2LDK,44.2,西,東京都,...,〇,〇,〇,2年,15,鉄筋コンクリート,仲介手数料無料,"236,500円",,1
7,【空室31件】,ＯＲＳＵＳ戸越銀座,"112,000円","10,000円",122000円,107,1R,25.14,北東,東京都,...,〇,,〇,新築,6,鉄筋コンクリート,仲介手数料無料,"123,200円",,2
8,【空室4件】,ディームス大井町フォレスト,"94,000円","10,000円",104000円,514,1R,20.91,西,東京都,...,〇,,〇,2年,4,鉄筋コンクリート,仲介手数料無料,"103,400円",フリーレント1ヶ月(9.4万円お得),3
9,【空室3件】,プラウドフラット戸越公園,"104,000円","8,000円",112000円,220,1K,22.32,北東,東京都,...,〇,〇,〇,5年,5,鉄筋コンクリート,仲介手数料無料,"114,400円",フリーレント1ヶ月(10.4万円お得),3


In [101]:
#エクセルに落として重複がないか確認（確認をした後に非表示化）
#df.to_excel('output_file2.xlsx', index=False, engine='openpyxl')

Google スプレッドシートに情報を転送する

In [102]:
#必要なライブラリをインポート
import gspread
from google.auth import exceptions
from google.oauth2.service_account import Credentials

In [103]:
# サービスアカウント キーを使用して認証情報を作成
credentials = Credentials.from_service_account_file('smooth-ocean-406910-dc84e296dda8.json', scopes=['https://www.googleapis.com/auth/spreadsheets'])
gc = gspread.authorize(credentials)

In [104]:
# スプレッドシートのURLからワークシートを取得
spreadsheet_url = 'https://docs.google.com/spreadsheets/d/1JT6fZwPAl_LWajo7iqqCjH4MKsYYW5dXgNxQQxId7ew/edit'
worksheet = gc.open_by_url(spreadsheet_url).sheet1

In [105]:
# データフレームをGoogleスプレッドシートに書き込む
worksheet.clear()  # シートをクリア
worksheet.update([df.columns.values.tolist()] + df.values.tolist())

C:\Users\masay\AppData\Local\Temp\ipykernel_4456\4288850697.py:3: DeprecationWarning: [Deprecated][in version 6.0.0]: Method signature's arguments 'range_name' and 'values' will change their order. We recommend using named arguments for minimal impact. In addition, the argument 'values' will be mandatory of type: 'List[List]'. (ex) Worksheet.update(values = [[]], range_name=) 
  worksheet.update([df.columns.values.tolist()] + df.values.tolist())


{'spreadsheetId': '1JT6fZwPAl_LWajo7iqqCjH4MKsYYW5dXgNxQQxId7ew',
 'updatedRange': "'シート1'!A1:W21",
 'updatedRows': 21,
 'updatedColumns': 23,
 'updatedCells': 483}